In [ ]:
import torch
import torch.nn as nn
import os, time
import numpy as np
from model.net import FashionNet
from load_dataset import custom_dset, collate_fn, denorm
from torch.utils.data import DataLoader
import cv2
import argparse
from utils import AverageMeter, AverageAcc
from tqdm import tqdm
from focal_loss import *


In [ ]:
from IPython.display import display, HTML
import pandas as pd

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [3]:
def test(net, test_loader, accuracy):


    losses = AverageMeter()

    criterion = FocalLoss(gamma=2, alpha=0.25)
    label_map = { v:k for k,v in test_loader.dataset.label_map.items()} 

    bar = tqdm(total=len(test_loader))

    with torch.no_grad():
        for it, (imgs, labels, img_ids) in enumerate(test_loader):

            imgs = imgs.cuda()
            labels = labels.cuda()

            # run input through the network
            preds = net(imgs)
            loss = criterion(preds, labels)


            # measure accuracy and record loss
            accuracy.update(preds.data, labels.data, topk=(1, 5))
            losses.update(loss.item(), imgs.size(0))
            bar.update()
            if it % 100 == 0:
                # save to disk first batch of results for every epoch
                for i,im in enumerate(imgs):
                    im = im.data.cpu().numpy()
                    im = denorm(im.transpose(1,2,0))[...,::-1]

                    _, inds = torch.max(preds, 1)
                    pred = label_map[inds[i].item()]
                    gt_label = label_map[labels[i].item()]

                    fname =  str(img_ids[i]) + '__' 'Gt:' + gt_label + 'Pred:' + pred + '.jpg'
                    cv2.imwrite(os.path.join(out_path, fname),  im)

        bar.close()



In [ ]:
out_path = './data/show_test'
if not os.path.isdir(out_path):
    os.mkdir(out_path)


# check checkpoints folder for model_name
model_name = 'pnet_0.92.pth'

# get the current working directory
root = os.getcwd()

# load model
model_path = os.path.join(root, 'checkpoints', model_name)
model = torch.load(model_path)
label_map = model['label_map']
net = FashionNet(len(label_map))
net.load_state_dict(model['net'])

net = net.cuda()
net.eval()


test_pretrain = custom_dset('test_pretrain', label_map)
print('Pretrain Test Dataset of size: {}'.format(len(test_pretrain)))
test_loader = DataLoader(test_pretrain, batch_size=24, shuffle=False, collate_fn=collate_fn, num_workers=4)

accuracy = AverageAcc(label_map)
test(net, test_loader, accuracy)

# get accuracies per class and the averages 
topk_dict, top1, top5 = accuracy()



  0%|          | 0/520 [00:00<?, ?it/s]

Pretrain Test Dataset of size: 12476


 14%|█▎        | 71/520 [00:41<04:06,  1.82it/s]

In [ ]:

# collect accuracies into lists for printing purpose 
lc = [k for k in label_map]
t1 = [v[1] for k,v in topk_dict.items()]
t2 = [v[5] for k,v in topk_dict.items()]
avg = [round(top1, 2), round(top5, 2)]


In [ ]:
trainset =  custom_dset('train_pretrain')
lc_train = [trainset.freq[c] for c in lc]
# display train set statitstics 
df_dset = pd.DataFrame(np.vstack((lc, lc_train)).transpose(), index=['Trainset']*len(label_map),
                       columns=['Category', 'Data examples'])
display(df_dset)


# display dataset statitstics 
df_dset = pd.DataFrame(np.vstack( (lc, [v[1] for _,v in accuracy.count.items()])).transpose(), columns=['Category', 'Data examples'],
                       index=['Testset']*len(label_map))
display(df_dset)

# concatenate accuracy lists in numpy array of shape (num_classes, num_columns)
df_class = pd.DataFrame(np.vstack( (lc, t1, t2)).transpose(), columns=['Category', 'Top1', 'Top5'], index=['TestAcc']*len(label_map))
df_avg = pd.DataFrame(np.array(avg).reshape(1,2), columns=['Top1', 'Top5'], index=['Average'])

# display the accuracies 
display(df_class)
display(df_avg)


In [ ]:
top1
